In [8]:
import pandas as pd;
import math as mt;
import numpy;

            

Coordenadas Montes Claros - MG
Latitude: 16° 43' 41" / -16.6855037 Sul
Longitude: 43° 51' 54" / -43.8276535 Oeste
Altitude: 638 Metros

In [12]:
datum_parametros = pd.read_csv("datum_parametros.csv")
display(datum_parametros)

,id,datum,semi_eixo,achat,deltax,deltay,deltaz
0,0,NaN,0.0,0.000000,0.00,0.00,0.00
1,1,SAD69,6378160.0,0.003353,-67.35,3.88,-38.22
2,2,CorregoAlegre,6378388.0,0.003367,-206.05,168.28,-3.82
3,3,AstroChua,6378388.0,0.003367,-144.35,243.37,-33.22
4,4,WGS84,6378137.0,0.003353,0.00,0.00,0.00
5,5,SIRGAS2000,6378137.0,0.003353,0.00,0.00,0.00


In [ ]:
# Função que converte coordenadas geográficas para UTM
# return x = Longitude UTM (Metros)
# return y = Latitude UTM (Metros)
# zona UTM 
def geo_to_utm(lon, lat, semi_eixo, achat, hemis, lon_mc):
    pi = pi();
    
    if hemis == "norte":
        offy = 0.;
    else:
        offy = 10000000.;
        
    #converte lat long em UTM
    k0 = 1. - (1. / 2500.);
    equad = 2. *achat - pow(achat,(float)(2));
    elinquad = equad / (1. - equad);
    
    aux1 = equad * equad;
    aux2 = aux1 * equad;
    aux3 = mt.sin(((float)(2)) * lat);
    aux4 = mt.sin(((float)(4)) * lat);
    aux5 = mt.sin(((float)(6)) * lat);
    aux6 = (1. - equad / 4. - 3. * aux1/64. - 5. * aux2 / 256.) * lat;
    aux7 = (3. * equad / 8. + 3. * aux1 / 32. + 45. * aux2 / 1024.) * aux3;
    aux8 = (15. * aux1 / 256. + 45. * aux2 / 1024.) * aux4;
    aux9 = (35. * aux2 / 3072.) * aux5;
    
    n = semi_eixo / (mt.sqrt(((float)(1)) - equad * pow(mt.sin(lat),(float)(2))));
    t = pow(mt.tan(lat),(float)(2));
    c = elinquad * pow(mt.cos(lat),(float)(2));
    ag = (lon - lon_mc) * mt.cos(lat);
    m = semi_eixo * (aux6 - aux7 - aux8 - aux9);
    
    aux10 = (1. - t + c) * ag * ag * ag / 6.;
    aux11 = (5. - 18. * t + t * t + 72. * c - 58. * elinquad) * (pow(ag,(float)(5))) / 120.;
    aux12 = (5. - t + 9. * c + 4. * c * c) * ag * ag * ag * ag / 24.;
    aux13 = (61.-58. * t + t * t + 600. * c- 330. * elinquad) * (pow(ag,(float)(6))) / 720.;
    x = 500000. + k0 * n * (ag + aux10 + aux11);
    y = offy + k0 * (m + n * (mt.tan(lat))*(ag * ag / 2. + aux12 + aux13));
    
    print("Lat UTM: ", x);
    print("Lon UTM: ", y);

In [ ]:
# Função que converte coordenadas UTM para coordenadas Geográficas
# return x = Longitude em coordenadas geográficas (Graus decimais)
# return y = Latitude em coordenadas geográficas (Graus Decimais)

def utm_to_geo(x,y,semi_eixo,achat,hemis,lon_mc):

	# Define PI
	pi = pi();

	# verifica hemisferio
	if hemis == "norte":
		y = y + 10000000;
			
	# Converte UTM para Lat/Long;
	k0 = 1. - (1. / 2500.);
	equad = 2. * achat - pow(achat, (float)(2));
	elinquad = equad / (1. - equad);
	e1 = (1. - mt.sqrt((float)(1) - equad)) / (1. + mt.sqrt((float)(1) - equad));

	aux1 = equad * equad;
	aux2 = aux1 * equad;
	aux3 = e1 * e1;
	aux4 = e1 * aux3;
	aux5 = aux4 * e1;

	m = (y - 10000000.) / k0;
	mi = m / (semi_eixo*(1. -equad /4. - 3. * aux1 / 64. - 5. * aux2 / 256.));

	aux6 = (3. * e1/2. - 27. * aux4/32.) * mt.sin((float)(2) * mi );
	aux7 = (21. * aux3/16. - 55. * aux5/32.) * mt.sin((float)(4) * mi);
	aux8 = (151. * aux4/96.) * mt.sin((float)(6) * mi);

	lat1 = mi + aux6 + aux7 + aux8;
	c1 = elinquad * pow(mt.cos(lat1),(float)(2));
	t1 = pow(mt.tan(lat1),(float)(2));
	n1 = semi_eixo / mt.sqrt((float)(1)-equad * pow(mt.sin(lat1),(float)(2)));
	quoc = pow(((float)(1)-equad * mt.sin(lat1) * mt.sin(lat1)),(float)(3));
	r1 = semi_eixo * (1.-equad) / mt.sqrt(quoc);
	d = (x - 500000.) / (n1 * k0);

	aux9 = (5. + 3. * t1 + 10. * c1 - 4. * c1 * c1 - 9. * elinquad) * d * d * d * d / 24.;
	aux10 = (61. + 90. * t1 + 298. * c1 + 45. * t1 * t1 - 252. * elinquad - 3. * c1 * c1) * pow(d,(float)(6)) / 720.;
	aux11 = d - (1. + 2. * t1 + c1) * d * d * d / 6.;
	aux12 = (5. - 2. * c1 + 28. * t1 - 3. * c1 * c1 + 8. * elinquad + 24. * t1 * t1) * pow(d,(float)(5)) / 120.;

	lat = lat1 - (n1*mt.tan(lat1)/r1)*(d*d/2. - aux9 + aux10);
	lon = lon_mc + (aux11 + aux12)/mt.cos(lat1);

	print("Lat Geo: ", lat);
	print("Lon Geo: ", lon);
    

In [9]:
# Função que define o meridiando Central da Zona UTM com base na Longitude informada (necessita revisão)
# @param lon = longitude em coordenadas geográficas (graus decimais).
def define_mer_cent(lon):
    
        if lon != 0.:
            sinal = (int)(lon/abs(lon))
        else:
            mc1_utm = 3.;
            mc2_utm = -3.;
        
        ind1=0
        k=0
        ind2=6
        
        while((abs(lon))>((float)(ind2))):
            mc = ind1 + 3.;
            k+=1;
            ind1 = 6 * k;
            ind2 = ind1 + 6;
        
        if ((abs(lon)) < ((float)(mc))) and (lon != 0.):
            mc1_utm = mc2_utm = ((float)(mc));
            print("Entrou no 1° if!!!!");
        
        if ((abs(lon)) > ((float)(mc))) and ((abs(lon)) != ((float)(mc) + 3.)):
            mc1_utm = mc2_utm = ((float)(mc));
            print("Entrou no 2° if!!!!");
        
        if ((abs(lon)) == ((float)(mc) + 3.)) and ((abs(lon)) != 180.):
            mc1_utm = ((float)(mc));
            mc2_utm = ((float)(mc)) + 6.;
            print("Entrou no 3° if!!!!");
        
        if (lon == 180.):
            mc1_utm = 177.;
            mc2_utm = -177.;
            print("Entrou no 4° if!!!!");
        
        if (lon == -108.):
            mc1_utm = -177.;
            mc2_utm = 177.;
            print("Entrou no 5° if!!!!");
        
        if ((abs(lon)) == ((float)(mc))):
            mc1_utm = mc2_utm = ((float)(mc));
            print("Entrou no 6° if!!!!");
        
        if (lon != 0.) and ((abs(lon)) != 180):
            mc1_utm *= sinal;
            mc2_utm *= sinal;
            print("Entrou no 7° if!!!!");
            
        print("MC: ", mc)
        print("MC1_UTM: ", mc2_utm)
        print("MC2_UTM: ", mc2_utm)
            
            
        

In [10]:
#main code

define_mer_cent(-16.6855037)

Entrou no 2° if!!!!
Entrou no 7° if!!!!
MC:  9.0
MC1_UTM:  -9.0
MC2_UTM:  -9.0
